encoder transformer层的linear1层（etl1）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import matmul as mm
import math_util as mu
from NNutils import *
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
nbits = 4

In [3]:
linear_name = "etl1"
linear_name_full = "ex_linear1"
feedback_bits = 256
ncodebooks = 64 # max:64
ncentroids = 128
# if method == METHOD_MITHRAL:
#     ncentroids = 16
train_sam_num = 125 # 训练集样本数
test_sam_num = 1000 # 测试集样本数
batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    linearin_path_train= ''
    linearout_path_train= ''
    linearin_path_test = ''
    linearout_path_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_joined = '/data/hdr/transformer_data/joined'
    dir_train = os.path.join(dir_joined, 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join(dir_joined, 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    linearin_path_train= 'ex_linear1in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'ex_linear1_y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    linearout_path_train= 'ex_linear1out_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    linearin_path_test = 'ex_linear1in_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
    linearout_path_test = 'ex_linear1out_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
else:
    raise NameError("You are running the script in a new computer %s, please define dirs" % host_name)


weightpath = 'ex_linear1_w_f256.npy'
biaspath = 'ex_linear1_b_f256.npy'
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileNotFoundError:
    os.makedirs(dir_result)
except FileExistsError:
    pass 


In [5]:
dataset_prepare(dir_joined, linear_name_full, feedback_bits, [train_sam_num, test_sam_num], batch_size, S1 = S1)

In [6]:
est3 = mm.estFactory(X_path=linearin_path_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method], nbits=nbits)

running method:  Mithral
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    1065.870246439427
learn_multisplits(): returning loss:  0.2679029136779718
learn_multisplits(): initial loss:    651.5780927184801
learn_multisplits(): returning loss:  0.1894425149075687
learn_multisplits(): initial loss:    649.2333590561892
learn_multisplits(): returning loss:  0.16227918659569696
learn_multisplits(): initial loss:    936.8366025805473
learn_multisplits(): returning loss:  0.2119341142824851
learn_multisplits(): initial loss:    959.7519711827034
learn_multisplits(): returning loss:  0.2610050281509757
learn_multisplits(): initial loss:    712.6405246181769
learn_multisplits(): returning loss:  0.32374924002215266
learn_multisplits(): initial loss:    1245.1493607120306
learn_multisplits(): returning loss:  0.5568850063718855
learn_multisplits(): initial loss:    2115.3592320266293
learn_multisplits(): returning loss:  1.427807778120041
learn_multisplits(): init

In [7]:
x_test = np.load(dir_test+'/'+linearin_path_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [8]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1]) #AMM字典模式需要复原y大小
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_EXACT:
    train_sam_num = 0 # 训练集样本数
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), y_out_last_re.astype(np.float32))
else:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)

[[-0.48835498 -0.25156578 -0.47112092 ... -0.24716164  1.256324
  -0.17062312]
 [-0.36311036 -0.25156578 -0.22063169 ... -0.12191702  1.256324
  -0.17062312]
 [-0.23786576 -0.3768104  -0.47112092 ... -0.37240624  1.256324
  -0.29586774]
 ...
 [-0.11262114 -0.12632117 -0.47112092 ... -0.12191702  0.63010097
  -0.17062312]
 [-0.36311036 -0.25156578 -0.47112092 ... -0.12191702  0.8805902
  -0.42111236]
 [-0.11262114 -0.25156578 -0.09538707 ... -0.12191702  0.63010097
  -0.17062312]]
y_out_last.shape:  (1024000, 512)
y_out_last_re.shape:  (1000, 32, 32, 512)
